In [1]:
# ref: https://www.kaggle.com/jacklinggu/lstm-with-glove-embedding-public-lb-score-0-049

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
from models_def import cnn2d,cnn_v1,cnn_v2,cudnn_gru,lstm_v1,cnn_gru,gru_v1
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.preprocessing import text, sequence

model_defines = [
    [cnn2d,'cnn2d'],
    [cnn_v1,'cnn_v1'],
    [cnn_v2,'cnn_v2'],
    [cudnn_gru,'cudnn_gru'],
    [lstm_v1,'lstm_v1'],
    [cnn_gru,'cnn_gru'],
    [gru_v1,'gru_v1']
]

max_features = 40000
maxlen = 150

def clean_text( text ):
    text = text.lower().split()
    text = " ".join(text)
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+\-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    #
    return text

train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")

list_sentences_train = train["comment_text"].fillna("CVxTz").apply(clean_text).values
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train[list_classes].values
list_sentences_test = test["comment_text"].fillna("CVxTz").apply(clean_text).values
print(y.shape)

Using TensorFlow backend.


(159571, 6)


In [2]:
tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(list_sentences_train))
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
X_train = sequence.pad_sequences(list_tokenized_train, maxlen=maxlen)
X_test = sequence.pad_sequences(list_tokenized_test, maxlen=maxlen)

print(X_train.shape,X_test.shape)

# check word_index
tmp_cnt = 0
for k in tokenizer.word_index:
    print(k,tokenizer.word_index[k])
    tmp_cnt += 1
    if tmp_cnt >5:
        break
word_idx = tokenizer.word_index

# read word2vec
word_vec_dict = {}
with open('../glove.840B.300d.txt') as f:
    for line in f:
        v_list = line.strip().split(' ')
        k = str(v_list[0])
        v = np.array([float(x) for x in v_list[1:]])
        word_vec_dict[k] = v

print(len(word_vec_dict))
print('Preparing embedding matrix')
EMBEDDING_DIM = 300
nb_words = min(max_features,len(word_idx))
embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))
for word,i in word_idx.items():
    if i >= max_features:
        continue
    else:
        if word in word_vec_dict:
            embedding_matrix[i] = word_vec_dict[word]
print('Null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))
del word_vec_dict

(159571, 150) (153164, 150)
nickct 45136
creacionismo 106870
infusers 89341
northamptonshire 44231
bookkeeping 38802
insignia 11659
2196007
Preparing embedding matrix
Null word embeddings: 3388


In [3]:
from sklearn.metrics import log_loss,accuracy_score
from sklearn.model_selection import KFold
import pickle
from keras.models import load_model

def eval_val(y,train_x):
    res = 0
    acc_res = 0
    for i in range(6):
        curr_loss = log_loss(y[:,i],train_x[:,i])
        acc = accuracy_score(y[:,i],train_x[:,i].round())
        print(i,curr_loss,acc)
        res += curr_loss
        acc_res += acc
    print('final',res/6, acc_res/6)

def kf_train(model_func,fold_cnt=3,rnd=1,epo=10,batch=64):
    kf = KFold(n_splits=fold_cnt, shuffle=False, random_state=233*rnd)
    train_pred, test_pred = np.zeros((159571,6)),np.zeros((153164,6))
    for train_index, test_index in kf.split(X_train):
        # x,y
        curr_x,curr_y = X_train[train_index],y[train_index]
        hold_out_x,hold_out_y = X_train[test_index],y[test_index]
        
        # model
        model = model_func(maxlen,nb_words,EMBEDDING_DIM,embedding_matrix,trainable_flag=True,comp=True)
        batch_size = batch
        epochs = epo
        file_path="best_model.h5"
        checkpoint = ModelCheckpoint(file_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
        callbacks_list = [checkpoint] 
        
        # train and pred
        model.fit(curr_x, curr_y, batch_size=batch_size, epochs=epochs, 
                  validation_data=(hold_out_x,hold_out_y), callbacks=callbacks_list)
        model = load_model(file_path)
        y_test = model.predict(X_test)
        test_pred += y_test
        hold_out_pred = model.predict(hold_out_x)
        train_pred[test_index] = hold_out_pred
    test_pred = test_pred / fold_cnt
    print('-------------------------------')
    eval_val(y,train_pred)
    return train_pred, test_pred

rnd_init = 1
for model_func,model_name in model_defines:
    for fold_k in [3,4]:
        print('==================================')
        if 'cnn' in model_name:
            epo = 8
            batch = 64
        else:
            epo = 10
            batch = 256
        train_pred,test_pred = kf_train(model_func,fold_k,rnd_init,epo,batch)
        output_feat = '../features/glove_{}_{}_feat.pkl'.format(model_name,fold_k)
        with open(output_feat,'wb') as fout:
            pickle.dump([train_pred,test_pred],fout)
        print(output_feat,'done')
        rnd_init += 1
        




Train on 106380 samples, validate on 53191 samples
Epoch 1/8
106380/106380 [==============================] - 269s 3ms/step - loss: 0.0583 - acc: 0.9796 - val_loss: 0.0443 - val_acc: 0.9828
Epoch 2/8
106380/106380 [==============================] - 136s 1ms/step - loss: 0.0428 - acc: 0.9832 - val_loss: 0.0441 - val_acc: 0.9828
Epoch 3/8
106380/106380 [==============================] - 136s 1ms/step - loss: 0.0364 - acc: 0.9853 - val_loss: 0.0446 - val_acc: 0.9831
Epoch 4/8
106380/106380 [==============================] - 136s 1ms/step - loss: 0.0318 - acc: 0.9869 - val_loss: 0.0473 - val_acc: 0.9825
Epoch 5/8
106380/106380 [==============================] - 137s 1ms/step - loss: 0.0279 - acc: 0.9884 - val_loss: 0.0521 - val_acc: 0.9827
Epoch 6/8
106380/106380 [==============================] - 137s 1ms/step - loss: 0.0247 - acc: 0.9899 - val_loss: 0.0600 - val_acc: 0.9822
Epoch 7/8
106380/106380 [==============================] - 137s 1ms/step - loss: 0.0223 - acc: 0.9910 - val_loss: 0

119678/119678 [==============================] - 143s 1ms/step - loss: 0.0281 - acc: 0.9884 - val_loss: 0.0529 - val_acc: 0.9820
Epoch 6/8
119678/119678 [==============================] - 143s 1ms/step - loss: 0.0249 - acc: 0.9897 - val_loss: 0.0599 - val_acc: 0.9819
Epoch 7/8
119678/119678 [==============================] - 143s 1ms/step - loss: 0.0224 - acc: 0.9909 - val_loss: 0.0664 - val_acc: 0.9818
Epoch 8/8
119678/119678 [==============================] - 143s 1ms/step - loss: 0.0200 - acc: 0.9919 - val_loss: 0.0708 - val_acc: 0.9815
Train on 119678 samples, validate on 39893 samples
Epoch 1/8
119678/119678 [==============================] - 144s 1ms/step - loss: 0.0560 - acc: 0.9800 - val_loss: 0.0454 - val_acc: 0.9822
Epoch 2/8
119678/119678 [==============================] - 144s 1ms/step - loss: 0.0425 - acc: 0.9834 - val_loss: 0.0431 - val_acc: 0.9835
Epoch 3/8
119678/119678 [==============================] - 143s 1ms/step - loss: 0.0364 - acc: 0.9853 - val_loss: 0.0458 - va

ResourceExhaustedError: OOM when allocating tensor with shape[64,150,300,1]
	 [[Node: dropout_16/cond/dropout/random_uniform/RandomUniform = RandomUniform[T=DT_INT32, dtype=DT_FLOAT, seed=87654321, seed2=977800, _device="/job:localhost/replica:0/task:0/gpu:0"](dropout_16/cond/dropout/Shape)]]
	 [[Node: loss_10/mul/_2693 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/gpu:0", send_device_incarnation=1, tensor_name="edge_726_loss_10/mul", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op 'dropout_16/cond/dropout/random_uniform/RandomUniform', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr